# Teleoperation

In this example we'll control the Jetbot remotely and take snapshots from the camera for training.

### Create gamepad controller

1. Download [https://www.x360ce.com/](https://www.x360ce.com/).  
2. Click on add button and chose keyboard as input device.
3. Remap the buttons by clicking on the controller.

Next, we'll create and display our controller using that index.

In [ ]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=1)  # replace with index of your controller

display(controller)

### Connect gamepad controller to robot motors

Now, even though we've connected our gamepad, we haven't yet attached the controls to our robot!  The first, and most simple control
we want to attach is the motor control.  We'll connect that to the left and right vertical axes using the ``dlink`` function.  The
``dlink`` function, unlike the ``link`` function, allows us to attach a transform between the ``source`` and ``target``.  Because
the controller axes are flipped from what we think is intuitive for the motor control, we'll use a small *lambda* function to
negate the value.

> WARNING: This next cell will move the robot if you touch the gamepad controller axes!

In [7]:
from jetbot import Robot
import traitlets

robot = Robot()

In [29]:
max_forward_speed = 0.7
max_turn_speed = 0.15
forward_speed = 0
turn_speed = 0
def diferential(x,axis):
    global forward_speed
    global turn_speed
    global max_turn_speed
    if axis == 'y':
        forward_speed = -max_forward_speed*x
        robot.left_motor.value = forward_speed + turn_speed
        robot.right_motor.value = forward_speed - turn_speed
    else:
        max_turn_speed = 0.4 if abs(forward_speed) < 0.5 else 0.15   
        turn_speed = max_turn_speed*x
        robot.left_motor.value = forward_speed + turn_speed
        robot.right_motor.value = forward_speed - turn_speed
    return 0

In [30]:
temp = widgets.FloatSlider()
left_link = traitlets.dlink((controller.axes[3], 'value'), (temp, 'value'), transform=lambda x:diferential(x,'y'))
right_link = traitlets.dlink((controller.axes[2], 'value'), (temp, 'value'), transform=lambda x:diferential(x,'x'))

-1.52587890625e-05
1.52587890625e-05


Awesome! Our robot should now respond to our gamepad controller movements.  Now we want to view the live video feed from the camera!

### Create camera instance

Well, right now there's no image presented, because we haven't set the value yet!  We can do this by creating our ``Camera``
class and attaching the ``value`` attribute of the camera to the ``value attribute of the image.

First, let's create the camera instance, we call the ``instance`` method which will create a new camera
if it hasn't been created yet.  If once already exists, this method will return the existing camera.

In [ ]:
from jetbot import Camera

camera = Camera.instance()

### Connect Camera to Image widget

Our camera class currently only produces values in BGR8 (blue, green, red, 8bit) format, while our image widget accepts values in compressed *JPEG*.
To connect the camera to the image we need to insert the ``bgr8_to_jpeg`` function as a transform in the link.  We do this below

In [22]:
from jetbot import bgr8_to_jpeg

image = widgets.Image(format='jpeg', width=300, height=300)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

### Stop robot if network disconnects

You can drive your robot around by looking through the video feed. But what if your robot disconnects from Wifi?  Well, the motors would keep moving and it would keep trying to stream video and motor commands.  Let's make it so that we stop the robot and unlink the camera and motors when a disconnect occurs.

In [23]:
from jetbot import Heartbeat


def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        left_link.unlink()
        right_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)

# attach the callback function to heartbeat status
heartbeat.observe(handle_heartbeat_status, names='status')

### Save snapshots with gamepad button

Now, we'd like to be able to save some images from our robot.  Let's make it so the right bumper (index 5) saves a snapshot of the current live image.  We'll save the images in the ``snapshots/`` directory, with a name that is guaranteed to be unique using the ``uuid`` python package.  We use the ``uuid1`` identifier, because this also encodes the date and MAC address which we might want to use later.

In [24]:
import uuid
import subprocess

subprocess.call(['mkdir', '-p', 'snapshots'])

snapshot_image = widgets.Image(format='jpeg', width=300, height=300)

def save_snapshot(change):
    # save snapshot when button is pressed down
    if change['new']:
        file_path = 'snapshots/' + str(uuid.uuid1()) + '.jpg'
        
        # write snapshot to file (we use image value instead of camera because it's already in JPEG format)
        with open(file_path, 'wb') as f:
            f.write(image.value)
            
        # display snapshot that was saved
        snapshot_image.value = image.value


controller.buttons[5].observe(save_snapshot, names='value')

display(widgets.HBox([image, snapshot_image]))
display(controller)

Controller(axes=(Axis(value=1.52587890625e-05), Axis(value=-1.52587890625e-05), Axis(value=1.52587890625e-05),…

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/jetbot-0.3.0-py3.6.egg/jetbot/heartbeat.py", line 35, in _run
    self.status = Heartbeat.Status.dead
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 588, in __set__
    self.set(obj, value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 577, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1209, in _notify_trait
    type='change',
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1214, in notify_change
    return self._notify_observers(change)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py",

### Unlink the controller from the motors

In [ ]:
left_link.unlink()
right_link.unlink()

### Remove the camera to use it in other notebooks

In [ ]:
camera_link.unlink()
camera.stop()